In [ ]:
import pickle
import gzip
import numpy as np
import os
import time
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.optim as optim
from matplotlib import pyplot as plt
from torchsummary import summary
import albumentations as A
from albumentations.pytorch import ToTensorV2
import random
import torch.nn.functional as F

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [ ]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

def save_zipped_pickle(obj, filename):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, 2)

In [ ]:
class TestDataset(Dataset):
    
    def __init__(self, images, transform=None):
        self.transform = transform
        self.images = images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        
        image = self.images[:, :, idx]
        
        transformed = self.transform(image=image)
        image = transformed["image"]
            
        return torch.div(image, 255).float()

In [ ]:
test_to_tensor = A.Compose(
    [
        A.Resize(128, 128),
        A.augmentations.transforms.CLAHE(p=1),
        ToTensorV2()
    ]
)

In [ ]:
test_data = load_zipped_pickle("data/test.pkl")

In [ ]:
# need to load model (unet) as well
# code is not tested

In [ ]:
# make prediction for test
predictions = []
for d in test_data:

    prediction = np.array(np.zeros_like(d['video']), dtype=bool)
    height = prediction.shape[0]
    width = prediction.shape[1]

    test_to_original = A.Compose([A.Resize(height, width)])

    video = d['video']
    test_dataset = TestDataset(video, test_to_tensor)
    test_loader = DataLoader(test_dataset, batch_size=16, pin_memory=True)

    with torch.no_grad():
        out = torch.Tensor().to(device)
        for x in test_loader:
            x = x.to(device)
            torch.cat((out, unet(x)))

    for i in range(out.shape[0]):
        out[i] = test_to_original(image=out[i])["image"]

    out = (out > 0.5).cpu().numpy()
    out = out.transpose(1, 2, 0)

    prediction[int(height/2)-50:int(height/2+50), int(width/2)-50:int(width/2+50)] = out[int(height/2)-50:int(height/2+50), int(width/2)-50:int(width/2+50)]
    
    # DATA Strucure
    predictions.append({
        'name': d['name'],
        'prediction': prediction
        }
    )

In [ ]:
# save in correct format
save_zipped_pickle(predictions, 'data/my_predictions.pkl')